This tutorial shows the potential of using CO2 capture resources in Macro such as Natural Gas Power CCS, SMR H2 CCS. Natural Gas Power CCS and SMR H2 CCS works by specifying a CO2 capture rate (for example 0.9) of the total CO2 emitted by natural gas used in producing power and H2 which effectively reduces the total emissions by 90%. There are a total of 2 CO2 node types: 1) Captured CO2 sink node, where its input is for captured CO2 from Natural Gas Power CCS and SMR H2 CCS, 2) Emission CO2 sink node to account for emission by fossil sources.

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [2]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [3]:
T = 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [4]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(NaturalGas => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Electricity => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1)), PeriodLength = 8760)

In [5]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;
ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas

ngcc_ccs_inv_cost = 66400;# $/MW
ngcc_ccs_fom_cost = 12287.0;# $/MW
ngcc_ccs_vom_cost = 3.65; #$/MW
ngcc_ccs_capsize = 250.0;
ngcc_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas
ngcc_ccs_CO2_captured_rate = 0.9; #Tonnes of CO2 captured/Tonness of CO2 produced by fuel

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MW of H2
electrolyzer_vom_cost = 0.0;

smr_inv_cost = 1033333/H2_MWh;# $/MW of H2
smr_fom_cost = 20500/H2_MWh;# $/MW of H2
smr_vom_cost = 0.0; #$/MW of H2
smr_capsize = 20*H2_MWh;
smr_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas

smr_ccs_inv_cost = 1133333/H2_MWh;# $/MW of H2
smr_ccs_fom_cost = 21500/H2_MWh;# $/MW of H2
smr_ccs_vom_cost = 0.0; #$/MW of H2
smr_ccs_capsize = 20*H2_MWh;
smr_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas
smr_ccs_CO2_captured_rate = 0.9; #Tonnes of CO2 captured/Tons of CO2 produced by fuel


In [6]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [7]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [8]:
solar_pv = Resource{Electricity}(;
    id = :Solar_PV,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    capacity_factor = solar_capacity_factor,
    can_expand = true, 
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = solar_inv_cost,
    fixed_om_cost = solar_fom_cost,
    constraints = [Macro.CapacityConstraint()]
)

Resource{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :Solar_PV, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.3834, 0.2594, 0.078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, true, false, 85300.0, 18760.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

We can also assigne a battery storage to the node:

In [9]:
battery = SymmetricStorage{Electricity}(;
    id = :battery,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    can_expand = true,
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = battery_inv_cost,
    investment_cost_storage = battery_inv_cost_storage,
    fixed_om_cost = battery_fom_cost,
    fixed_om_cost_storage = battery_fom_cost_storage,
    variable_om_cost = battery_vom_cost,
    variable_om_cost_storage = battery_vom_cost_storage,
    efficiency_injection = battery_eff_down,
    efficiency_withdrawal = battery_eff_up,
    constraints = [Macro.CapacityConstraint(),Macro.StorageCapacityConstraint()]
)

SymmetricStorage{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :battery, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0.0, Inf, 0.0, Inf, 0.0, 0.0, true, false, 19584.0, 22494.0, 0.0, 4895.0, 5622.0, 0.0, 0.15, 0.15, 0.0, 0.92, 0.92, 0.0, 1.0, 10.0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing), Macro.StorageCapacityConstraint(missing, missing, missing)])

We also consider a hydrogen demand node:

In [10]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [11]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    demand = zeros(length(time_interval(NaturalGas))),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

In [12]:
ng_source = Resource{NaturalGas}(;
id = :NG_Source,
node = ng_node,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
can_expand = false, 
can_retire = false,
price = ng_fuel_price,
existing_capacity = Inf,
)

Resource{NaturalGas}(Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :NG_Source, Float64[], 1.0, [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264  …  14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, Inf, false, false, 0.0, 0.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We define an atmospheric CO2 node (which is also going to provide input int DAC with CO2 from the atmosphere to capture):

In [13]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_sink = Sink{CO2}(;
node = co2_node,
id = Symbol("CO2_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

Sink{CO2}(Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_sink, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We define captured CO2 node and sink to account for captured CO2. Instead of a sink, we could also define a storage resource!

In [14]:
co2_captured_node = Node{CO2}(;
id = Symbol("CO2_captured_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_captured_sink = Sink{CO2}(;
node = co2_captured_node,
id = Symbol("CO2_captured_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

Sink{CO2}(Node{CO2}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_captured_sink, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

This transformation has 3 edges: each edge carries a specific commodity flow

In [15]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
number_of_stoichiometry_balances = 2,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_heatrate,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_fuel_CO2]
))

push!(ngcc.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 65400.0, 10287.0, 3.55, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{N

We are now ready to define a natural gas plant with CCS! Note that we require three stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas $\times$ (1-CCS rate) = CO2

Fuel_CO2 $\times$ NaturalGas $\times$ CCS rate = CO2_Captured

This transformation has 4 edges: each edge carries a specific commodity flow

In [16]:
ngcc_ccs = Transformation{NaturalGasPowerCCS}(;
id = :NGCC_CCS,
time_interval = time_interval(Electricity),
number_of_stoichiometry_balances = 3,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc_ccs.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc_ccs,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_ccs_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_ccs_heatrate,0.0,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_ccs_inv_cost,
fixed_om_cost = ngcc_ccs_fom_cost,
variable_om_cost = ngcc_ccs_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_ccs_fuel_CO2*(1-ngcc_ccs_CO2_captured_rate),ngcc_ccs_fuel_CO2*ngcc_ccs_CO2_captured_rate]
))

push!(ngcc_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(ngcc_ccs.TEdges,TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPowerCCS}(:NGCC_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 66400.0, 12287.0, 3.65, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [17]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                number_of_stoichiometry_balances = 1,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(electrolyzer.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [1.0],
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(electrolyzer.TEdges,TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = [electrolyzer_efficiency]
))

2-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 66.66, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0], 0.0, Inf, 0.0, 61006.090609060906, 915.0915091509152, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConst

We are now ready to define a natural gas hydrogen plant or SMR similar to a natural gas power plant

In [18]:
smr = Transformation{NaturalGasHydrogen}(;
                id = :SMR,
                time_interval = time_interval(Hydrogen),
                number_of_stoichiometry_balances = 2,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_heatrate,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_inv_cost,
    fixed_om_cost = smr_fom_cost,
    variable_om_cost = smr_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_fuel_CO2]
))

push!(smr.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(:SMR, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 31003.090309030904, 615.0615061506151, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, An

Now, let's add a SMR hydrogen plant with CCS, similar to a natural gas power plant with CCS

In [19]:
smr_ccs = Transformation{NaturalGasHydrogenCCS}(;
                id = :SMR_CCS,
                time_interval = time_interval(Hydrogen),
                number_of_stoichiometry_balances = 3,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr_ccs.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_ccs_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_ccs_heatrate,0.0,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_ccs_inv_cost,
    fixed_om_cost = smr_ccs_fom_cost,
    variable_om_cost = smr_ccs_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_ccs_fuel_CO2*(1-smr_ccs_CO2_captured_rate),smr_ccs_fuel_CO2*smr_ccs_CO2_captured_rate]
))

push!(smr_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(smr_ccs.TEdges,TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogenCCS}(:SMR_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 34003.390339033904, 645.0645064506451, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), 

Now that we have implemented different technologies, we can create the JuMP model.

In [20]:
components = [solar_pv; battery; ngcc; ngcc_ccs; electrolyzer; smr; smr_ccs;ng_source;co2_sink;co2_captured_sink];
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;components];

In [21]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [22]:
add_planning_variables!.(components,model);

In [23]:
add_operation_variables!.(system, model);

In [24]:
add_all_model_constraints!.(system, model);

In [25]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [26]:
println(ngcc.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] == 0


In [27]:
println(ngcc.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [28]:
println(ngcc_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] == 0


In [29]:
println(ngcc_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [30]:
println(ngcc_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2_Captured[5] == 0


In [31]:
println(electrolyzer.constraints[1].constraint_ref[1,5])

vFLOW_Hydrogen_H2[5] - 0.7406666666666667 vFLOW_Electricity_E[5] == 0


In [32]:
println(smr.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] == 0


In [33]:
println(smr.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [34]:
println(smr_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] == 0


In [35]:
println(smr_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] == 0


In [36]:
println(smr_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2_Captured[5] == 0


In [37]:
using Gurobi

In [38]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-03-14


In [39]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 262810 rows, 245305 columns and 643737 nonzeros
Model fingerprint: 0x2de177a4
Coefficient statistics:
  Matrix range     [1e-04, 7e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 2e+04]
Presolve removed 170979 rows and 170986 columns
Presolve time: 0.20s
Presolved: 91831 rows, 74319 columns, 240453 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.02s

Barrier statistics:
 Dense cols : 8
 AA' NZ     : 1.837e+05
 Factor NZ  : 1.447e+06 (roughly 80 MB of memory)
 Factor Ops : 2.363e+07 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual      

The installed electrolyzer capacity is:

In [40]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

0.0

The installed SMR capacity is

In [41]:
Macro.value(Macro.capacity(smr.TEdges[1]))

1039.5627

The installed SMR CCS capacity is

In [42]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

0.0

The installed solar capacity in MW is:

In [43]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity in MW is:

In [44]:
Macro.value(Macro.capacity(battery))

100.0

The installed NGCC capacity in MW is:

In [45]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

16617.0

The installed NGCC CCS capacity in MW is:

In [46]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [47]:
base_emissions  = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

3.42974550073138e7

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [48]:
base_CO2_captured  = Macro.value(sum(Macro.withdrawal(co2_captured_sink)[t] for t in time_interval(CO2)))

0.0

Lets add a low CO2 cap:

In [49]:
Macro.@constraint(model,CO2Cap,sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)) <= 1000000);

In [50]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 262811 rows, 245305 columns and 652497 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 7e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 1e+06]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5077394e+09   3.329746e+07   0.000000e+00      0s
    1130    3.5077397e+09   4.601430e+08   0.000000e+00      5s
    2130    3.5077402e+09   4.580668e+08   0.000000e+00     10s
    3130    3.5077405e+09   4.561049e+08   0.000000e+00     16s
    4030    3.5077409e+09   4.542695e+08   0.000000e+00     20s
    4890    3.5169019e+09   4.397877e+08   0.000000e+00     25s
    5850    3.5169028e+09   4.106202e+08   0.000000e+00     31s
    6650    3.5169031e+09   3.82731

Now, the electrolyzer capacity is:

In [51]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

900.2433

The SMR capacity is:

In [52]:
Macro.value(Macro.capacity(smr.TEdges[1]))

0.0

The SMR CCS capacity is:

In [53]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

856.9143

Now the solar capacity is:

In [54]:
Macro.value(Macro.capacity(solar_pv))

46181.923101621614

The installed battery capacity is:

In [55]:
Macro.value(Macro.capacity(battery))

8665.940727046356

The installed NG capacity is:

In [56]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

0.0

The installed NG CCS capacity is:

In [57]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

7035.059272953645

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [58]:
Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

999999.9999999995

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [59]:
Macro.value(sum(Macro.withdrawal(co2_captured_sink)[t] for t in time_interval(CO2)))

9.000000000000091e6

We see that in a low CO2 emission case, Power and H2 CCS are deployed along with PV and electrolyzers

However, if we use a net-zero CO2 cap:

In [60]:
Macro.@constraint(model,CO2CapZero,sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)) <= 0);

In [61]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 262812 rows, 245305 columns and 661257 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 7e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 1e+06]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7363527e+09   1.000000e+06   0.000000e+00      0s
    1090    1.2425592e+10   1.013135e+09   0.000000e+00      5s
    2070    1.2588632e+10   1.187324e+09   0.000000e+00     11s
    2820    1.2750150e+10   1.621414e+08   0.000000e+00     15s
    3490    1.2913745e+10   4.537869e+08   0.000000e+00     20s
    4250    1.3032481e+10   1.303419e+08   0.000000e+00     25s
    4980    1.3191152e+10   1.965745e+09   0.000000e+00     30s
    5740    1.3282790e+10   4.22604

Now, the electrolyzer capacity is:

In [62]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

1039.5627

The SMR capacity is:

In [63]:
Macro.value(Macro.capacity(smr.TEdges[1]))

0.0

The SMR CCS capacity is:

In [64]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

0.0

Now the solar capacity is:

In [65]:
Macro.value(Macro.capacity(solar_pv))

133116.22350205318

The installed battery capacity is:

In [66]:
Macro.value(Macro.capacity(battery))

16372.85

The installed NG capacity is:

In [67]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

0.0

The installed NG CCS capacity is:

In [68]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [69]:
Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

0.0

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [70]:
Macro.value(sum(Macro.withdrawal(co2_captured_sink)[t] for t in time_interval(CO2)))

0.0

We see that in a net-zero emission case, only PV power and electrolyzer H2 are deployed.